# 다중선형회귀분석

In [1]:
# 데이터 구성:Series, DataFrame
import pandas as pd
# 행렬 연산
import numpy as np
# 데이터 시각화
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
# 회귀분석
from statsmodels.formula.api import ols
# qqplot, 회귀분석의 상수항 추가
from statsmodels.api import qqplot, add_constant
# 선형 모델 formula(y ~ X1 + X2 + ...)
import statsmodels.formula.api as smf
# 다중공선성 확인
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 학습용/평가용 데이터 분리:train, test
from sklearn.model_selection import train_test_split
# 선형회귀모형
from sklearn.linear_model import LinearRegression
# 변수 선택법(후진제거법)
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler
# 평가함수
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score

In [2]:
df_raw = pd.read_csv('add_variable_customer.csv', encoding='euc-kr')

In [3]:
df_raw.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
df_raw

,CustomerId,Gender,Age,AgeGroup,Married,Dependents,noDependents,Referrals,noReferrals,PaperlessBilling,...,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,TenureMonths,AvgCharge,TotalCharge,TotalRevenue,MonthRoamCharge,AvgExtraDataCharge
0,C-10001,여성,37,30,Yes,No,0,Yes,2,Yes,...,3,0,47910,431100,21,83590.476190,1755400,2186500,20528,0
1,C-10002,남성,46,40,No,No,0,No,0,No,...,5,11300,12090,108710,21,68928.571429,1447500,1567510,5176,538
2,C-10003,남성,50,50,No,No,0,No,0,Yes,...,1,0,38080,152100,4,98180.000000,490900,643000,38025,0
3,C-10004,남성,78,70,Yes,No,0,Yes,1,Yes,...,1,0,31410,408720,13,112742.857143,1578400,1987120,31440,0
4,C-10005,여성,75,70,Yes,No,0,Yes,3,Yes,...,1,0,8360,24970,3,94500.000000,378000,402970,8323,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6858,C-17039,여성,20,20,No,No,0,No,0,No,...,4,0,52770,685680,25,64428.000000,1610700,2296380,27427,0
6859,C-17040,남성,40,40,Yes,No,0,Yes,1,Yes,...,1,0,18310,402730,22,101891.304348,2343500,2746230,18305,0
6860,C-17041,남성,22,20,No,No,0,No,0,Yes,...,5,0,21020,42040,14,56600.000000,792400,834440,3002,0
6861,C-17042,남성,21,20,No,No,0,Yes,5,No,...,3,0,2370,160460,79,80107.594937,6328500,6488960,2031,0


In [5]:
##### @Gender : 여성(1), 남성(0)

df_raw['Gender'] = df_raw['Gender'].replace(to_replace='여성', value=1)
df_raw['Gender'] = df_raw['Gender'].replace(to_replace='남성', value=0)

##### @Married : Yes(1), No(0)

df_raw['Married'] = df_raw['Married'].replace(to_replace='Yes', value=1)
df_raw['Married'] = df_raw['Married'].replace(to_replace='No', value=0)

##### @Referrals : Yes(1), No(0)

df_raw['Referrals'] = df_raw['Referrals'].replace(to_replace='Yes', value=1)
df_raw['Referrals'] = df_raw['Referrals'].replace(to_replace='No', value=0)

##### @Dependents: Yes(1), No(0)

df_raw['Dependents'] = df_raw['Dependents'].replace(to_replace='Yes', value=1)
df_raw['Dependents'] = df_raw['Dependents'].replace(to_replace='No', value=0)

##### @PaperlessBilling  : Yes(1), No(0)

df_raw['PaperlessBilling'] = df_raw['PaperlessBilling'].replace(to_replace='Yes', value=1)
df_raw['PaperlessBilling'] = df_raw['PaperlessBilling'].replace(to_replace='No', value=0)

##### @PaymentMethod

df_raw['PaymentMethod'] = df_raw['PaymentMethod'].replace(to_replace='신용카드', value=1)
df_raw['PaymentMethod'] = df_raw['PaymentMethod'].replace(to_replace='계좌이체', value=0)
df_raw['PaymentMethod'] = df_raw['PaymentMethod'].replace(to_replace='이체/메일확인', value=2)

##### @OnlineSecurity

df_raw['OnlineSecurity'] = df_raw['OnlineSecurity'].replace(to_replace='Yes', value=1)
df_raw['OnlineSecurity'] = df_raw['OnlineSecurity'].replace(to_replace='No', value=0)

##### @OnlineBackup  : Yes(1), No(0)

df_raw['OnlineBackup'] = df_raw['OnlineBackup'].replace(to_replace='Yes', value=1)
df_raw['OnlineBackup'] = df_raw['OnlineBackup'].replace(to_replace='No', value=0)

##### @TechSupport  : Yes(1), No(0)

df_raw['TechSupport'] = df_raw['TechSupport'].replace(to_replace='Yes', value=1)
df_raw['TechSupport'] = df_raw['TechSupport'].replace(to_replace='No', value=0)

##### @UnlimitedData : Yes(1), No(0)

df_raw['UnlimitedData'] = df_raw['UnlimitedData'].replace(to_replace='Yes', value=1)
df_raw['UnlimitedData'] = df_raw['UnlimitedData'].replace(to_replace='No', value=0)

In [33]:
df_raw

,CustomerId,Gender,Age,AgeGroup,Married,Dependents,noDependents,Referrals,noReferrals,PaperlessBilling,...,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,TenureMonths,AvgCharge,TotalCharge,TotalRevenue,MonthRoamCharge,AvgExtraDataCharge
0,C-10001,1,37,30,1,0,0,1,2,1,...,3,0,47910,431100,21,83590.476190,1755400,2186500,20528,0
1,C-10002,0,46,40,0,0,0,0,0,0,...,5,11300,12090,108710,21,68928.571429,1447500,1567510,5176,538
2,C-10003,0,50,50,0,0,0,0,0,1,...,1,0,38080,152100,4,98180.000000,490900,643000,38025,0
3,C-10004,0,78,70,1,0,0,1,1,1,...,1,0,31410,408720,13,112742.857143,1578400,1987120,31440,0
4,C-10005,1,75,70,1,0,0,1,3,1,...,1,0,8360,24970,3,94500.000000,378000,402970,8323,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6858,C-17039,1,20,20,0,0,0,0,0,0,...,4,0,52770,685680,25,64428.000000,1610700,2296380,27427,0
6859,C-17040,0,40,40,1,0,0,1,1,1,...,1,0,18310,402730,22,101891.304348,2343500,2746230,18305,0
6860,C-17041,0,22,20,0,0,0,0,0,1,...,5,0,21020,42040,14,56600.000000,792400,834440,3002,0
6861,C-17042,0,21,20,0,0,0,1,5,0,...,3,0,2370,160460,79,80107.594937,6328500,6488960,2031,0


In [6]:
df_raw['StartDate'] = pd.to_datetime(df_raw['StartDate'])
df_raw = df_raw[(df_raw["StartDate"].dt.year>=2020)]

In [7]:
df_raw.drop(['CustomerId', 'StartDate', 'EndDate', 'ChurnLabel',
             'EndDateTmp', 'ChurnCategory', 'ChurnReason'], axis=1, inplace=True)

/var/folders/_0/hhcfgww90217hcgb7pdwcnhm0000gn/T/ipykernel_19602/2268371761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw.drop(['CustomerId', 'StartDate', 'EndDate', 'ChurnLabel',


In [8]:
df_raw

,Gender,Age,AgeGroup,Married,Dependents,noDependents,Referrals,noReferrals,PaperlessBilling,PaymentMethod,...,SatisScore,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,TenureMonths,AvgCharge,TotalCharge,TotalRevenue,MonthRoamCharge,AvgExtraDataCharge
0,1,37,30,1,0,0,1,2,1,1,...,3,0,47910,431100,21,83590.476190,1755400,2186500,20528,0
1,0,46,40,0,0,0,0,0,0,1,...,5,11300,12090,108710,21,68928.571429,1447500,1567510,5176,538
2,0,50,50,0,0,0,0,0,1,0,...,1,0,38080,152100,4,98180.000000,490900,643000,38025,0
3,0,78,70,1,0,0,1,1,1,0,...,1,0,31410,408720,13,112742.857143,1578400,1987120,31440,0
4,1,75,70,1,0,0,1,3,1,1,...,1,0,8360,24970,3,94500.000000,378000,402970,8323,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6853,0,24,20,0,0,0,0,0,0,1,...,4,0,55940,55940,13,22930.769231,298100,354040,4303,0
6855,1,20,20,0,0,0,0,0,1,1,...,3,0,41250,288600,19,105905.263158,2012200,2300800,15189,0
6856,1,53,50,0,0,0,0,0,1,1,...,1,0,47570,47570,1,77550.000000,155100,202670,47570,0
6857,1,36,30,0,0,0,0,0,0,0,...,1,0,2260,9040,4,23660.000000,118300,127340,2260,0


## 데이터 분할

In [9]:
# 설명변수, 목표변수 데이터 구분
df_raw_x = df_raw.drop('SatisScore', axis=1, inplace=False)
df_raw_y = df_raw['SatisScore']

In [10]:
# 변수명 저장
v_feature_names = df_raw_x.columns
# StandardScaler 적용
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df_raw_x)
df_scaled = pd.DataFrame(df_scaled, columns=v_feature_names)
df_scaled.head()

,Gender,Age,AgeGroup,Married,Dependents,noDependents,Referrals,noReferrals,PaperlessBilling,PaymentMethod,...,CustomerLTV,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,TenureMonths,AvgCharge,TotalCharge,TotalRevenue,MonthRoamCharge,AvgExtraDataCharge
0,1.0,0.283333,0.166667,1.0,0.0,0.0,1.0,0.2,1.0,0.5,...,0.050563,0.000000,0.847965,0.350807,0.869565,0.523660,0.573543,0.554732,0.364780,0.000000
1,0.0,0.433333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.853817,0.066667,0.213982,0.088463,0.869565,0.401368,0.470452,0.394322,0.091977,0.005951
2,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.619775,0.000000,0.673982,0.123771,0.130435,0.645348,0.150166,0.154737,0.675700,0.000000
3,0.0,0.966667,0.833333,1.0,0.0,0.0,1.0,0.1,1.0,0.0,...,0.428285,0.000000,0.555929,0.332596,0.521739,0.766815,0.514280,0.503063,0.558685,0.000000
4,1.0,0.916667,0.833333,1.0,0.0,0.0,1.0,0.3,1.0,0.5,...,0.365707,0.000000,0.147965,0.020319,0.086957,0.614654,0.112365,0.092534,0.147899,0.000000


In [11]:
ss_df =pd.concat([df_raw_y, df_scaled], axis=1)

In [12]:
ss_df

,SatisScore,Gender,Age,AgeGroup,Married,Dependents,noDependents,Referrals,noReferrals,PaperlessBilling,...,CustomerLTV,TotalExtraDataCharge,AvgRoamCharge,TotalRoamCharge,TenureMonths,AvgCharge,TotalCharge,TotalRevenue,MonthRoamCharge,AvgExtraDataCharge
0,3.0,1.0,0.283333,0.166667,1.0,0.0,0.0,1.0,0.2,1.0,...,0.050563,0.000000,0.847965,0.350807,0.869565,0.523660,0.573543,0.554732,0.364780,0.000000
1,5.0,0.0,0.433333,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.853817,0.066667,0.213982,0.088463,0.869565,0.401368,0.470452,0.394322,0.091977,0.005951
2,1.0,0.0,0.500000,0.500000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.619775,0.000000,0.673982,0.123771,0.130435,0.645348,0.150166,0.154737,0.675700,0.000000
3,1.0,0.0,0.966667,0.833333,1.0,0.0,0.0,1.0,0.1,1.0,...,0.428285,0.000000,0.555929,0.332596,0.521739,0.766815,0.514280,0.503063,0.558685,0.000000
4,1.0,1.0,0.916667,0.833333,1.0,0.0,0.0,1.0,0.3,1.0,...,0.365707,0.000000,0.147965,0.020319,0.086957,0.614654,0.112365,0.092534,0.147899,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,NaN,0.0,0.883333,0.833333,0.0,0.0,0.0,0.0,0.0,1.0,...,0.430538,0.000000,0.200000,0.083124,0.347826,0.637091,0.311213,0.266443,0.201688,0.000000
2161,NaN,0.0,0.066667,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.620025,0.000000,0.990088,0.045521,0.521739,0.017708,0.085613,0.079854,0.076464,0.000000
2162,NaN,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.886608,0.000000,0.730088,0.234848,0.782609,0.709783,0.659524,0.584353,0.269907,0.000000
2163,NaN,1.0,0.550000,0.500000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.943680,0.000000,0.841947,0.038710,0.000000,0.473277,0.037734,0.040627,0.845313,0.000000


In [13]:
# train/test 분리
ss_df_train, ss_df_test = train_test_split(ss_df, test_size = 0.3, random_state = 42) 
print("train data size : {}".format(ss_df_train.shape))
print("test data size : {}".format(ss_df_test.shape))

train data size : (2538, 27)
test data size : (1088, 27)


In [14]:
ss_df_train.columns

Index(['SatisScore', 'Gender', 'Age', 'AgeGroup', 'Married', 'Dependents',
       'noDependents', 'Referrals', 'noReferrals', 'PaperlessBilling',
       'PaymentMethod', 'OnlineSecurity', 'OnlineBackup', 'TechSupport',
       'UnlimitedData', 'AvgDownloadGB', 'ChurnScore', 'CustomerLTV',
       'TotalExtraDataCharge', 'AvgRoamCharge', 'TotalRoamCharge',
       'TenureMonths', 'AvgCharge', 'TotalCharge', 'TotalRevenue',
       'MonthRoamCharge', 'AvgExtraDataCharge'],
      dtype='object')

In [15]:
ss_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3626 entries, 0 to 2164
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SatisScore            2166 non-null   float64
 1   Gender                2166 non-null   float64
 2   Age                   2166 non-null   float64
 3   AgeGroup              2166 non-null   float64
 4   Married               2166 non-null   float64
 5   Dependents            2166 non-null   float64
 6   noDependents          2166 non-null   float64
 7   Referrals             2166 non-null   float64
 8   noReferrals           2166 non-null   float64
 9   PaperlessBilling      2166 non-null   float64
 10  PaymentMethod         2166 non-null   float64
 11  OnlineSecurity        2166 non-null   float64
 12  OnlineBackup          2166 non-null   float64
 13  TechSupport           2166 non-null   float64
 14  UnlimitedData         2166 non-null   float64
 15  AvgDownloadGB        

In [16]:
# sns.pairplot(ss_df)

In [80]:
reg_ss_model = smf.ols(formula = "SatisScore ~ Married + TotalRoamCharge " , data = ss_df_train)

# 적합
reg_ss_result = reg_ss_model.fit()
print(reg_ss_result.summary())

                            OLS Regression Results                            
Dep. Variable:             SatisScore   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     5.686
Date:                Thu, 17 Mar 2022   Prob (F-statistic):            0.00363
Time:                        03:28:21   Log-Likelihood:                -791.02
No. Observations:                 468   AIC:                             1588.
Df Residuals:                     465   BIC:                             1600.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           2.9342      0.086     